In [1]:
import os
import sys
sys.path.append(os.path.join(os.getcwd().replace("notebooks/distilled_models", "")))

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from config import Config
from data.dataset import NetworkDataset, load_datasets
from model_config import LSTM_Models
from compact.knowledge_distillation import KnowledgeDistillation

In [2]:
conf = Config()
load_models = LSTM_Models()
teacher_model = load_models.lstm_4
teacher_model.load()
student_model = load_models.light_lstm_4
kd = KnowledgeDistillation(
    teacher=teacher_model,
    student=student_model,
    device=conf.device,
    distillation="feature",
    rnn=True
)

Checkpoint loaded from /global/D1/homes/jorgetf/Network-Packet-ML-Model/checkpoint/large_model/lstm_4.pth!


In [3]:
X_train, y_train, X_val, y_val, X_test, y_test = load_datasets(conf.datasets, "lstm")

# create train, val and test dataloaders
train_dataset = NetworkDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, conf.batch_size, shuffle=True)

val_dataset = NetworkDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, conf.batch_size, shuffle=True)

test_dataset = NetworkDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, conf.batch_size)

In [4]:
data, labels = next(iter(train_loader))
print(data.shape, labels.shape)

torch.Size([32, 513, 1]) torch.Size([32])


In [5]:
acc, train_loss, val_loss = kd.train_kd(train_loader, val_loader, conf.epochs)

Epoch: 1/10, Accuracy: 92.61%, Train loss: 0.0148, Val loss: 0.1002
Epoch: 2/10, Accuracy: 94.65%, Train loss: 0.0051, Val loss: 0.0753
Epoch: 3/10, Accuracy: 95.02%, Train loss: 0.0043, Val loss: 0.0687
Epoch: 4/10, Accuracy: 95.30%, Train loss: 0.0039, Val loss: 0.0625
Epoch: 5/10, Accuracy: 95.61%, Train loss: 0.0036, Val loss: 0.0590
Epoch: 6/10, Accuracy: 95.73%, Train loss: 0.0033, Val loss: 0.0550
Epoch: 7/10, Accuracy: 96.01%, Train loss: 0.0032, Val loss: 0.0533
Epoch: 8/10, Accuracy: 96.13%, Train loss: 0.0030, Val loss: 0.0499
Epoch: 9/10, Accuracy: 95.94%, Train loss: 0.0030, Val loss: 0.0502
Epoch: 10/10, Accuracy: 96.13%, Train loss: 0.0029, Val loss: 0.0504


In [6]:
# test teacher accuracy
loss, accuracy = teacher_model.evaluate(test_loader)
print(f"Teacher accuracy: {100*accuracy:.2f}%")

Teacher accuracy: 96.64%


In [7]:
# test student accuracy
loss, accuracy = student_model.evaluate(test_loader)
print(f"Student accuracy: {100*accuracy:.2f}%")

Student accuracy: 96.18%


In [8]:
student_model.save()

Checkpoint saved at /global/D1/homes/jorgetf/Network-Packet-ML-Model/checkpoint/compressed_model/light_lstm_4.pth
